In [2]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 18.4 MB/s eta 0:00:0000:0100:01


In [2]:
import cv2
import os
import numpy as np

if cv2 and os:
    print('success')

success


In [3]:
video_path = 'compressed_final.mp4'
output_dir = 'raw_clips'
os.makedirs(output_dir, exist_ok=False)

FileExistsError: [Errno 17] File exists: 'raw_clips'

In [11]:
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
clip_length = 1
frame_count = 0
clip_count = 300
start_time = 200

In [12]:
cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

# Loop to extract clips
while cap.isOpened() and clip_count < 600:  # Extract next 300 clips
    frames = []
    for _ in range(int(fps * clip_length)):  # Extract frames for each clip
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    if frames:  # Check if any frames were captured
        clip_path = os.path.join(output_dir, f"clip_{clip_count:03d}.mp4")
        height, width, _ = frames[0].shape  # Get frame dimensions
        out = cv2.VideoWriter(
            clip_path, 
            cv2.VideoWriter_fourcc(*'mp4v'), 
            fps, 
            (width, height)
        )
        for frame in frames:
            out.write(frame)
        out.release()  # Release the writer
        clip_count += 1

    if not ret:
        break

cap.release()
print(f"Extracted {clip_count - 300} new clips from {video_path}")

Extracted 300 new clips from THE CRAZIEST FINAL.mp4


In [13]:
import csv

In [7]:
clip_dir = "raw_clips"
output_csv = "labels.csv"

# Load existing labels into a set to avoid re-labeling
labeled_clips = set()
if os.path.exists(output_csv):
    with open(output_csv, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            labeled_clips.add(row[0])  # Add file_name to the set

# Open the CSV file in append mode to add new labels
with open(output_csv, mode='a', newline='') as file:
    writer = csv.writer(file)

    # Sort the clips in alphabetical/numerical order for consistency
    clip_names = sorted(os.listdir(clip_dir))

    # Loop through each clip, skipping already labeled ones
    for clip_name in clip_names:
        if clip_name in labeled_clips:
            continue  # Skip already labeled clips

        clip_path = os.path.join(clip_dir, clip_name)

        # Open and play each clip for labeling
        cap = cv2.VideoCapture(clip_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            cv2.imshow("Clip", frame)
            if cv2.waitKey(30) & 0xFF == ord('q'):  # Press 'q' to move to the next clip
                break

        cap.release()
        cv2.destroyAllWindows()

        # Ask for label input (type "pass" or "dribble")
        label = input(f"Enter label for {clip_name} (pass/dribble): ").strip().lower()

        # Write the file name and label to the CSV
        writer.writerow([clip_name, label])

print("Labeling complete. New labels added to", output_csv)

Labeling complete. New labels added to labels.csv


In [8]:
import pandas as pd

df = pd.read_csv(output_csv)
df.shape, df.head(), df['label'].unique()


((300, 2),
       file_name    label
 0  clip_000.mp4     pass
 1  clip_001.mp4  dribble
 2  clip_002.mp4     pass
 3  clip_003.mp4     pass
 4  clip_004.mp4  dribble,
 array(['pass', 'dribble', 'none'], dtype=object))

In [9]:
df = df.drop(df[df['label'] == 'none'].index)
df.shape

(231, 2)